In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    prev_actions = tf.placeholder(tf.int32, [None], name='prev_actions')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return prev_actions, states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        actions_logits = tf.layers.dense(inputs=nl2, units=action_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return actions_logits, Qlogits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size,
               prev_actions, states, actions, next_states, rewards, dones, rates):
    ################################################ a, Q = act(s, a')
    prev_actions_labels = tf.one_hot(indices=prev_actions, depth=action_size, dtype=states.dtype)
    actions_logits, aQs_logits = Act(states=states, actions=prev_actions_labels, hidden_size=hidden_size, 
                                     action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    aQs_logits = tf.reshape(aQs_logits, shape=[-1])
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    ################################################ s', Q = env(s, a)
    next_states_logits, eQs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                         action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits,
                                                                   labels=next_states_labels))
    eQs_logits = tf.reshape(eQs_logits, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    ###################################################### eQ = aQ
    eloss2 = tf.reduce_mean(tf.square(eQs_logits - aQs_logits))
    aloss2 = -tf.reduce_mean(aQs_logits)
    return actions_logits, aloss, eloss, aloss2, eloss2

In [7]:
def model_opt(a_loss, e_loss, e_loss2, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
        e_opt2 = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss2, var_list=e_vars)
    return a_opt, e_opt, a_opt2, e_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.prev_actions, self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2, self.e_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            prev_actions=self.prev_actions, states=self.states, actions=self.actions, # model input
            next_states=self.next_states, rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2, self.e_opt2 = model_opt(a_loss=self.a_loss, 
                                                                     e_loss=self.e_loss,
                                                                     a_loss2=self.a_loss2, 
                                                                     e_loss2=self.e_loss2,
                                                                     a_learning_rate=a_learning_rate,
                                                                     e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [23]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size

In [24]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [25]:
state = env.reset()
total_reward = 0
num_step = 0
prev_action = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([prev_action, state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    prev_action = action
    if done is True:
        #print(done)
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0
        num_step = 0
        prev_action = 0

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list, eloss2_list = [], [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(1111):
        aloss_batch, eloss_batch, aloss2_batch, eloss2_batch = [], [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1
        prev_action = 0

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                feed_dict={model.states: state.reshape([1, -1]), model.prev_actions: np.reshape(prev_action, [1])}
                action_logits = sess.run(model.actions_logits, feed_dict)
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([prev_action, state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            prev_action = action
            
            # Training minibatch
            while True:
                batch = memory.sample(batch_size=batch_size)
                prev_actions = np.array([each[0] for each in batch])
                states = np.array([each[1] for each in batch])
                actions = np.array([each[2] for each in batch])
                next_states = np.array([each[3] for each in batch])
                rewards = np.array([each[4] for each in batch])
                dones = np.array([each[5] for each in batch])
                rates = np.array([each[6] for each in batch])
                if np.min(rates) > 0:
                    break
            feed_dict = {model.prev_actions: prev_actions,
                         model.states: states, 
                         model.actions: actions,
                         model.next_states: next_states,
                         model.rewards: rewards,
                         model.dones: dones,
                         model.rates: rates}
            aloss, _, eloss, _ = sess.run([model.a_loss, model.a_opt, model.e_loss, model.e_opt], feed_dict)
            aloss2, _, eloss2, _ = sess.run([model.a_loss2, model.a_opt2, model.e_loss2, model.e_opt2], feed_dict)
            #print(aloss2, eloss2)
            #print(prev_actions.shape, prev_actions.dtype)
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            eloss2_batch.append(eloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'eloss2:{:.4f}'.format(np.mean(eloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        eloss2_list.append([ep, np.mean(eloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:34.0000 R:34.0000 rate:0.0680 aloss:1.3727 eloss:1.3987 aloss2:0.1342 eloss2:0.0796 exploreP:0.9966
Episode:1 meanR:21.5000 R:9.0000 rate:0.0180 aloss:1.3724 eloss:1.3935 aloss2:0.1335 eloss2:0.0739 exploreP:0.9958
Episode:2 meanR:20.0000 R:17.0000 rate:0.0340 aloss:1.3737 eloss:1.3899 aloss2:0.1336 eloss2:0.0719 exploreP:0.9941
Episode:3 meanR:18.0000 R:12.0000 rate:0.0240 aloss:1.3786 eloss:1.3871 aloss2:0.1330 eloss2:0.0689 exploreP:0.9929
Episode:4 meanR:26.6000 R:61.0000 rate:0.1220 aloss:1.3746 eloss:1.3794 aloss2:0.1331 eloss2:0.0618 exploreP:0.9869
Episode:5 meanR:24.6667 R:15.0000 rate:0.0300 aloss:1.3739 eloss:1.3711 aloss2:0.1336 eloss2:0.0548 exploreP:0.9855
Episode:6 meanR:23.0000 R:13.0000 rate:0.0260 aloss:1.3745 eloss:1.3703 aloss2:0.1332 eloss2:0.0547 exploreP:0.9842
Episode:7 meanR:23.3750 R:26.0000 rate:0.0520 aloss:1.3754 eloss:1.3659 aloss2:0.1321 eloss2:0.0518 exploreP:0.9817
Episode:8 meanR:22.5556 R:16.0000 rate:0.0320 aloss:1.3744 eloss:1.3631 a

Episode:71 meanR:22.5972 R:17.0000 rate:0.0340 aloss:1.3778 eloss:1.3069 aloss2:0.1300 eloss2:0.0209 exploreP:0.8513
Episode:72 meanR:22.5479 R:19.0000 rate:0.0380 aloss:1.3747 eloss:1.3068 aloss2:0.1295 eloss2:0.0213 exploreP:0.8498
Episode:73 meanR:23.1081 R:64.0000 rate:0.1280 aloss:1.3755 eloss:1.3066 aloss2:0.1293 eloss2:0.0211 exploreP:0.8444
Episode:74 meanR:23.0133 R:16.0000 rate:0.0320 aloss:1.3772 eloss:1.3063 aloss2:0.1292 eloss2:0.0207 exploreP:0.8431
Episode:75 meanR:23.1711 R:35.0000 rate:0.0700 aloss:1.3780 eloss:1.3065 aloss2:0.1288 eloss2:0.0210 exploreP:0.8401
Episode:76 meanR:23.0909 R:17.0000 rate:0.0340 aloss:1.3733 eloss:1.3062 aloss2:0.1291 eloss2:0.0211 exploreP:0.8387
Episode:77 meanR:23.2179 R:33.0000 rate:0.0660 aloss:1.3753 eloss:1.3062 aloss2:0.1293 eloss2:0.0207 exploreP:0.8360
Episode:78 meanR:23.0633 R:11.0000 rate:0.0220 aloss:1.3734 eloss:1.3061 aloss2:0.1294 eloss2:0.0205 exploreP:0.8351
Episode:79 meanR:22.9125 R:11.0000 rate:0.0220 aloss:1.3810 elos

Episode:141 meanR:23.8700 R:17.0000 rate:0.0340 aloss:1.3740 eloss:1.3036 aloss2:0.1272 eloss2:0.0190 exploreP:0.7198
Episode:142 meanR:24.2200 R:49.0000 rate:0.0980 aloss:1.3779 eloss:1.3038 aloss2:0.1263 eloss2:0.0190 exploreP:0.7163
Episode:143 meanR:23.8900 R:16.0000 rate:0.0320 aloss:1.3722 eloss:1.3035 aloss2:0.1261 eloss2:0.0193 exploreP:0.7152
Episode:144 meanR:24.0000 R:21.0000 rate:0.0420 aloss:1.3735 eloss:1.3038 aloss2:0.1270 eloss2:0.0188 exploreP:0.7137
Episode:145 meanR:24.1800 R:46.0000 rate:0.0920 aloss:1.3738 eloss:1.3036 aloss2:0.1270 eloss2:0.0190 exploreP:0.7105
Episode:146 meanR:24.4100 R:56.0000 rate:0.1120 aloss:1.3793 eloss:1.3034 aloss2:0.1273 eloss2:0.0189 exploreP:0.7066
Episode:147 meanR:24.2700 R:18.0000 rate:0.0360 aloss:1.3769 eloss:1.3036 aloss2:0.1264 eloss2:0.0193 exploreP:0.7053
Episode:148 meanR:24.1200 R:13.0000 rate:0.0260 aloss:1.3751 eloss:1.3035 aloss2:0.1274 eloss2:0.0188 exploreP:0.7044
Episode:149 meanR:24.2000 R:37.0000 rate:0.0740 aloss:1.

Episode:211 meanR:31.1600 R:17.0000 rate:0.0340 aloss:1.3805 eloss:1.3022 aloss2:0.1225 eloss2:0.0192 exploreP:0.5724
Episode:212 meanR:31.3500 R:30.0000 rate:0.0600 aloss:1.3800 eloss:1.3021 aloss2:0.1225 eloss2:0.0191 exploreP:0.5707
Episode:213 meanR:31.9400 R:79.0000 rate:0.1580 aloss:1.3767 eloss:1.3021 aloss2:0.1231 eloss2:0.0187 exploreP:0.5663
Episode:214 meanR:31.3000 R:20.0000 rate:0.0400 aloss:1.3754 eloss:1.3021 aloss2:0.1224 eloss2:0.0190 exploreP:0.5652
Episode:215 meanR:31.6000 R:47.0000 rate:0.0940 aloss:1.3763 eloss:1.3020 aloss2:0.1223 eloss2:0.0188 exploreP:0.5626
Episode:216 meanR:31.7100 R:27.0000 rate:0.0540 aloss:1.3782 eloss:1.3020 aloss2:0.1232 eloss2:0.0185 exploreP:0.5611
Episode:217 meanR:31.6600 R:10.0000 rate:0.0200 aloss:1.3789 eloss:1.3020 aloss2:0.1221 eloss2:0.0188 exploreP:0.5605
Episode:218 meanR:31.6600 R:16.0000 rate:0.0320 aloss:1.3813 eloss:1.3022 aloss2:0.1220 eloss2:0.0188 exploreP:0.5597
Episode:219 meanR:31.9400 R:51.0000 rate:0.1020 aloss:1.

Episode:281 meanR:32.8500 R:54.0000 rate:0.1080 aloss:1.3810 eloss:1.3017 aloss2:0.1181 eloss2:0.0176 exploreP:0.4578
Episode:282 meanR:32.8900 R:62.0000 rate:0.1240 aloss:1.3765 eloss:1.3017 aloss2:0.1174 eloss2:0.0177 exploreP:0.4550
Episode:283 meanR:32.9000 R:15.0000 rate:0.0300 aloss:1.3763 eloss:1.3017 aloss2:0.1180 eloss2:0.0174 exploreP:0.4543
Episode:284 meanR:32.7100 R:24.0000 rate:0.0480 aloss:1.3771 eloss:1.3016 aloss2:0.1178 eloss2:0.0177 exploreP:0.4533
Episode:285 meanR:32.5600 R:33.0000 rate:0.0660 aloss:1.3772 eloss:1.3018 aloss2:0.1169 eloss2:0.0178 exploreP:0.4518
Episode:286 meanR:32.7400 R:32.0000 rate:0.0640 aloss:1.3777 eloss:1.3018 aloss2:0.1172 eloss2:0.0178 exploreP:0.4504
Episode:287 meanR:32.6500 R:35.0000 rate:0.0700 aloss:1.3756 eloss:1.3016 aloss2:0.1179 eloss2:0.0176 exploreP:0.4489
Episode:288 meanR:32.2900 R:31.0000 rate:0.0620 aloss:1.3800 eloss:1.3016 aloss2:0.1175 eloss2:0.0175 exploreP:0.4475
Episode:289 meanR:32.1500 R:26.0000 rate:0.0520 aloss:1.

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
